In [1]:
import pytesseract

In [3]:
pwd

'/Users/admin-tascott/Documents/GitHub/eis_documents/enepa_repository/code'

In [19]:
base_dir = '../'
savedir = base_dir + 'enepa_repository/epa_comment_letters_ocr'
import os
import PyPDF2

def check_pdf_readability(savedir):
    readable_files = []
    ocr_needed_files = []

    for filename in os.listdir(savedir):
        if filename.lower().endswith('.pdf'):
            filepath = os.path.join(savedir, filename)
            try:
                with open(filepath, 'rb') as file:
                    pdf_reader = PyPDF2.PdfReader(file)
                    #sometimes EPA has an archive page on 1
                    # Check the first ten pages for extractable text
                    has_text = False
                    for i in range(min(10, len(pdf_reader.pages))):
                        page = pdf_reader.pages[i]
                        text = page.extract_text()
                        if text.strip():
                            has_text = True
                            break
                    if has_text:  # If there's extractable text in any of the first ten pages
                        readable_files.append(filename)
                    else:
                        ocr_needed_files.append(filename)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                ocr_needed_files.append(filename)  # Assume OCR is needed if there's an error

    return readable_files, ocr_needed_files

# Check the PDFs in both directories
machine_readable, need_ocr = check_pdf_readability(savedir)
machine_readable_air, need_ocr_air = check_pdf_readability(savedir_air)


In [21]:
len(machine_readable) + len(machine_readable_air) + len(need_ocr) + len(need_ocr_air)

450

In [23]:

print("Files with machine-readable text:")
for file in machine_readable + machine_readable_air:
    print(f"- {file}")

print("\nFiles that need OCR:")
for file in need_ocr + need_ocr_air:
    print(f"- {file}")

# Create directory for machine-readable RIAs
import os

machine_readable_dir = base_dir+'ria_analysis/ria_documents/epa_corpus/full_machine_readable_rias'
os.makedirs(machine_readable_dir, exist_ok=True)

print(f"\nCreated directory for machine-readable RIAs: {machine_readable_dir}")

import os
import json

def check_and_delete_corrupt_files(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith('.pdf'):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'rb') as file:
                    PyPDF2.PdfReader(file)  # Attempt to read the PDF file
            except PyPDF2.errors.PdfReadError:
                print(f"Corrupt file detected: {filename}")
                os.remove(file_path)
                print(f"Deleted: {filename}")

check_and_delete_corrupt_files(machine_readable_dir)

Files with machine-readable text:
- ZyPDF.cgi?Dockey=P100TFH1.PDF
- P10175J2.PDF?Dockey=P10175J2.PDF
- ZyPDF.cgi?Dockey=P100SFJ0.PDF
- ZyPDF.cgi?Dockey=P1004LNN.PDF
- ZyPDF.cgi?Dockey=P100U02B.PDF
- 20141125ria.pdf
- ZyPDF.cgi?Dockey=P10024CN.PDF
- gpo39643.pdf
- 111dproposalRIAfinal0602.pdf
- ZyPDF.cgi?Dockey=P1013ORN.PDF
- ZyPDF.cgi?Dockey=P10020UG.PDF
- ZyPDF.cgi?Dockey=P100LYPD.PDF
- ZyPDF.cgi?Dockey=P1000XC5.PDF
- fso2ria100602full.pdf
- ZyPDF.cgi?Dockey=P100FOHX.PDF
- ZyPDF.cgi?Dockey=P10190V0.PDF
- ZyPDF.cgi?Dockey=P10163WM.pdf
- gpo61946.pdf
- ZyPDF.cgi?Dockey=P1016CIF.pdf
- Zypdf.cgi?Dockey=P10160FS.pdf
- ZyPDF.cgi?Dockey=P10006MA.PDF
- ZyPDF.cgi?Dockey=P1017OW2.PDF
- oilgas_ria_nsps_final_2016-05.pdf
- ZyPDF.cgi?Dockey=P100Y2JY.PDF
- ZyPdf.cgi?Dockey=P1019OX8.PDF
- ZyPDF.cgi?Dockey=P100IXZI.PDF
- gpo14062.pdf
- gpo13080.pdf
- ZyPDF.cgi?Dockey=P10084SW.PDF
- ZyPDF.cgi?Dockey=P1005AUC.PDF
- ZyPDF.cgi?Dockey=P100Y6PS.PDF
- ZyPDF.cgi?Dockey=P1009I5W.PDF
- ZyPDF.cgi?Dockey=P1003L5

In [29]:

import shutil
from PIL import Image
import pytesseract
from pdf2image import convert_from_path

CLOBBER = False  # Set this to True if you want to overwrite existing files

def process_pdfs(savedir, machine_readable_dir, machine_readable, need_ocr):
    for filename in os.listdir(savedir):
        if filename.lower().endswith('.pdf'):
            source_path = os.path.join(savedir, filename)
            dest_path = os.path.join(machine_readable_dir, filename)
            
            # Check if the file already exists in the destination directory
            if os.path.exists(dest_path) and not CLOBBER:
                #print(f"Skipping {filename} as it already exists in {machine_readable_dir}")
                continue
            
            if filename in machine_readable:
                # Copy machine-readable PDFs directly
                shutil.copy2(source_path, dest_path)
                print(f"Copied {filename} to {machine_readable_dir}")
            elif filename in need_ocr:
                # Perform OCR on PDFs that need it
                try:
                    # Use ocrmypdf to create a new OCR'd copy of the PDF
                    if os.system(f"ocrmypdf --clean --clean-final --output-type pdf {source_path} {dest_path}") == 0:
                        print(f"OCR completed for {filename} and saved to {machine_readable_dir}")
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")

# Process the PDFs from both directories
process_pdfs(savedir, machine_readable_dir, machine_readable, need_ocr)
process_pdfs(savedir_air, machine_readable_dir, machine_readable_air, need_ocr_air)

print("\nAll files from both directories have been processed and saved to the machine-readable directory.")


All files from both directories have been processed and saved to the machine-readable directory.
